In [137]:
import geojson
import folium
import pandas as pd
import geopandas as gpd
import json

In [143]:
df = pd.read_excel('data.xlsx')

In [144]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LON, df.LAT), crs="EPSG:4326")

In [145]:
gdf = gdf.to_crs({'init': 'epsg:3857'})
gdf['geometry'] = gdf['geometry'].buffer(2000, 32)
gdf = gdf.to_crs({'init': 'epsg:4326'})

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [146]:
gdf['fill'] = '#ff0000'  # Прозрачная красная заливка
gdf['fill-opacity'] = 0.5  # 50% прозрачность
gdf['stroke'] = 'none'  # Нет обводки
gdf.to_file("output.geojson", driver="GeoJSON")

In [138]:
features = []
for _, row in gdf.iterrows():
    feature = {
        "type": "Feature",
        "geometry": json.loads(row['geometry'].to_json()),
        "properties": {
            "ADDRESS": row['ADDRESS'],
            "OBJ_NAME": row['OBJ_NAME']
        }
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open('circles.geojson', 'w') as f:
    json.dump(geojson, f)

AttributeError: 'Polygon' object has no attribute 'to_json'

In [134]:
m = folium.Map(location = [55.750066, 37.614356], zoom_start=10)

for _, r in gdf.iterrows():
    sim_geo = gpd.GeoSeries(gdf['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data = geo_j, style_function=lambda x: {'fillColor': 'orange'})

    geo_j.add_to(m)

    m.save('geo.html')

In [1]:
import geojson
import folium

# Table with points lat/lon, radius and lables.
data = [
    {'label': 'poin1','lat': 55.806690, 'lon': 37.577019, 'radius': 2},
    {'label': 'poin2','lat': 55.741420, 'lon': 37.693748, 'radius': 2}
]



# Create list from points
points = []
for item in data:
    point = geojson.Point((item['lon'], item['lat']))
    points.append(point)

# Create list circles from points list.
circles = []
for item in data:
    circle = geojson.Feature(
        geometry=geojson.Point((item['lon'], item['lat'])),
        properties={
            'radius': item['radius'],
            'color': 'red',
            'fillOpacity': 0.1,
            'stroke': 'red',
            'strokeWidth': 0.3,
            'label': item['label']
        }
    )
    circles.append(circle)

# Create GeoJSON
geojson_data = geojson.FeatureCollection(circles)

# Save GeoJSON to file
with open('circles.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

# Create folium map with circles
m = folium.Map(location=[55.7558, 37.6173], zoom_start=12)
for item in data:
    folium.Circle(
        location=[item['lat'], item['lon']],
        radius=item['radius']*1000,  # радиус в метрах
        color='red',
        fill=True,
        stroke=False,
        popup=item['label']
        # fillOpacity=0.1,
        # stroke=True,
        # strokeWeight=2,
    ).add_to(m)

# Save map to html file
m.save('circles.html')